In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
!pip install tensorflow-model-optimization

In [ ]:
!pip install tensorflow-addons

In [ ]:
!pip install PyYAML==5.1

In [ ]:
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

In [ ]:
# %cd '/content/drive/MyDrive/UpWork/Protein/'
# !git clone https://github.com/google/automl.git
# !mv automl/efficientdet .
# !rm -r automl
# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d0.tar.gz
# !tar zxf efficientdet-d0.tar.gz
# %cd /content

In [ ]:
# Split test images
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/data_preparation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/test_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/Karunakar_annotation_boxes \
  -o /content/drive/MyDrive/UpWork/Protein/Data/test_split_images16 \
  -l /content/drive/MyDrive/UpWork/Protein/Data/annotations_split16 \
  -d 768 -p 8 -f 0.18

In [ ]:
# Split validation images
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/data_preparation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/validation_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/Karunakar_annotation_boxes \
  -o /content/drive/MyDrive/UpWork/Protein/Data/validation_split_images16 \
  -l /content/drive/MyDrive/UpWork/Protein/Data/annotations_split16 \
  -d 768 -p 8 -f 0.18

In [ ]:
# Split train images
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/data_preparation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/train_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/Karunakar_annotation_boxes \
  -o /content/drive/MyDrive/UpWork/Protein/Data/train_split_images16 \
  -l /content/drive/MyDrive/UpWork/Protein/Data/annotations_split16 \
  -d 768 -p 8 -f 0.18

In [ ]:
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/coco_annotation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/train_split_images16/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations_split16 \
  -o /content/drive/MyDrive/UpWork/Protein/Data/train_split_annotation16.json
  
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/coco_annotation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/validation_split_images16/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations_split16 \
  -o /content/drive/MyDrive/UpWork/Protein/Data/validation_split_annotation16.json

!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/coco_annotation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/test_split_images16/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations_split16 \
  -o /content/drive/MyDrive/UpWork/Protein/Data/test_split_annotation16.json

In [ ]:
# Create TF Records
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/dataset/create_coco_tfrecord.py --logtostderr \
  --image_dir=/content/drive/MyDrive/UpWork/Protein/Data/train_split_images16 \
  --object_annotations_file=/content/drive/MyDrive/UpWork/Protein/Data/train_split_annotation16.json \
  --output_file_prefix=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_train_split16 \
  --num_shards=2

In [ ]:
# Create TF Records
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/dataset/create_coco_tfrecord.py --logtostderr \
  --image_dir=/content/drive/MyDrive/UpWork/Protein/Data/validation_split_images16 \
  --object_annotations_file=/content/drive/MyDrive/UpWork/Protein/Data/validation_split_annotation16.json \
  --output_file_prefix=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_validation_split16 \
  --num_shards=1

In [ ]:
# Create TF Records
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/dataset/create_coco_tfrecord.py --logtostderr \
  --image_dir=/content/drive/MyDrive/UpWork/Protein/Data/test_split_images16 \
  --object_annotations_file=/content/drive/MyDrive/UpWork/Protein/Data/test_split_annotation16.json \
  --output_file_prefix=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_test_split16 \
  --num_shards=1

**Training**

In [ ]:
# Training
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/main.py --mode=train_and_eval \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--train_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_train_split16-*-of-00002.tfrecord \
	--val_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_validation_split16-*-of-00001.tfrecord \
	--model_name=efficientdet-d0 \
	--ckpt=/content/drive/MyDrive/UpWork/Protein/efficientdet-d0 \
  --model_dir=/content/drive/MyDrive/UpWork/Protein/model_4/ \
	--train_batch_size=4 \
	--num_examples_per_epoch=743  --num_epochs=10  \
  --eval_batch_size=4 --eval_samples=190  \

In [ ]:
# Evalute on validation data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/main.py \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
  --mode=eval \
  --model_name=efficientdet-d0 \
  --model_dir=/content/drive/MyDrive/UpWork/Protein/model_2/	\
  --val_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_validation_split-*-of-00002.tfrecord \
  --eval_batch_size=4  --eval_samples=1220

In [ ]:
# Evalute on test data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/main.py \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
  --mode=eval \
  --model_name=efficientdet-d0 \
  --model_dir=/content/drive/MyDrive/UpWork/Protein/model_2/	\
  --val_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_test_split-*-of-00002.tfrecord \
  --eval_batch_size=4 --eval_samples=995

In [ ]:
# Exporting
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/model_inspect.py --runmode=saved_model \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--model_name=efficientdet-d0 \
  --ckpt_path=/content/drive/MyDrive/UpWork/Protein/model_4/model.ckpt-1114 \
  --saved_model_dir=/content/drive/MyDrive/UpWork/Protein/exported_model_4-6/	

In [ ]:
# Inference on test data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/model_inspect.py \
  --runmode=saved_model_infer \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--model_name=efficientdet-d0 \
  --saved_model_dir=/content/drive/MyDrive/UpWork/Protein/exported_model_2-4/	\
  --input_image=/content/drive/MyDrive/UpWork/Protein/Data/test_split_images/TcdA1-*_frames*.png \
  --output_image_dir=/content/drive/MyDrive/UpWork/Protein/output_test_model2_e4/ \
	--min_score_thresh=0.5 \
	--max_boxes_to_draw=150

In [ ]:
# Inference on training data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/model_inspect.py \
  --runmode=saved_model_infer \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--model_name=efficientdet-d0 \
  --saved_model_dir=/content/drive/MyDrive/UpWork/Protein/exported_model_1/	\
  --input_image=/content/drive/MyDrive/UpWork/Protein/train_images/TcdA1-0011_frames.png \
  --output_image_dir=output_train/ \
	--min_score_thresh=0.5 \
	--max_boxes_to_draw=50